# Variational Method Test

In [1]:
import numpy as np  
import functools
import itertools
import matplotlib.pyplot as plt 
from scipy   import  linalg as LA 
import json

from qiskit import IBMQ, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit import ParameterVector

from qiskit.utils                     import QuantumInstance
from qiskit.opflow                    import Z, I, X, Y
from qiskit.opflow         			  import PauliOp, SummedOp

from shor_function  import *
from pVQD			import *
from ansatze        import *
normalize = lambda x: x / np.linalg.norm(x)

/var/folders/1k/wfwgsdr57xz4qwc3p5r0l92h0000gq/T/ipykernel_8278/4011728452.py:13: DeprecationWarning: The ``qiskit.opflow`` module is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  from qiskit.opflow                    import Z, I, X, Y


In [2]:
potential = lambda x: x**2
true_sol = lambda x, t: normalize(1/np.sqrt(2)*np.pi**(-1/4)*(2*x**2 - 1)*np.exp(-x**2/2)*np.exp(-1j*5*t))
initial_wave_function = lambda x: true_sol(x, 0)

In [10]:
# Initialize system parameters for Time-dependent Schrodinger
n       = 6
N       = 2**n
L       = 5
dx      = 2*L/N
x_grid = np.arange(-L, L - dx/2, dx)
# g       = 1.0
dt      = 0.0
n_steps = 1

# Algorithm parameters

ths = 0.99999
depth = 3


### Example circ

ex_params = np.zeros((depth+1)*n +depth*(n-1))
wfn = hweff_ansatz(n,depth,ex_params)


### Shift
shift  = np.array(len(ex_params)*[0.01])

print("Initial shift:",shift)


### Generate the Hamiltonian
H = walsh_quantum_trotter(potential, initial_wave_function, n, L, dt, D=1/2, terms_kept=None, verbose=True, gate_count_only=False)

print(wfn)
# print(H)

### Backend
shots = 8000
backend  = Aer.get_backend('qasm_simulator')
instance = QuantumInstance(backend=backend,shots=shots)

# ### Prepare the observables to measure
obs = {}


Initial shift: [0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01]
     ┌───────┐ ░                                          ░ ┌───────┐ ░ »
q_0: ┤ Rx(0) ├─░──■───────────────────────────────────────░─┤ Ry(0) ├─░─»
     ├───────┤ ░  │ZZ(0)                                  ░ ├───────┤ ░ »
q_1: ┤ Rx(0) ├─░──■───────■───────────────────────────────░─┤ Ry(0) ├─░─»
     ├───────┤ ░          │ZZ(0)                          ░ ├───────┤ ░ »
q_2: ┤ Rx(0) ├─░──────────■───────■───────────────────────░─┤ Ry(0) ├─░─»
     ├───────┤ ░                  │ZZ(0)                  ░ ├───────┤ ░ »
q_3: ┤ Rx(0) ├─░──────────────────■───────■───────────────░─┤ Ry(0) ├─░─»
     ├───────┤ ░                          │ZZ(0)          ░ ├───────┤ ░ »
q_4: ┤ Rx(0) ├─░──────────────────────────■───────■───────░─┤ Ry(0) ├─░─»
     ├───────┤ ░                              

/var/folders/1k/wfwgsdr57xz4qwc3p5r0l92h0000gq/T/ipykernel_8278/713022558.py:38: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  instance = QuantumInstance(backend=backend,shots=shots)


In [12]:
### Initialize the algorithm

# Choose a specific set of parameters
initial_point = None

# Choose the gradient optimizer: 'sgd', 'adam'
opt  = 'sgd'
# Choose how to estimate the gradient on hardware: 'param_shift', 'spsa'
grad = 'param_shift'
# Choose which type of cost function use: 'global', 'local'
cost = 'local'


algo = pVQD(hamiltonian   = H,
			ansatz        = hweff_ansatz,
			ansatz_reps   = depth,
			parameters    = ex_params,
			initial_shift = shift,
			instance      = instance,
			shots         = shots)

algo.run(ths,dt,n_steps, 
	     obs_dict      = obs,
	     filename      = 'data/test_results.dat',
	     max_iter      = 1000,
	     opt           = opt,
	     cost_fun      = cost,
	     grad          = grad,
	     initial_point = initial_point,
         is_circuit    = True)

Running the algorithm


Time slice: 1
Shift before optimizing this step: [0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01]
Initial parameters: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Shift optimizing step: 1
Overlap [0.50625    0.00180431]
Gradient [ 8.22916667e-04  6.14583333e-04 -2.39583333e-04  1.11458333e-03
 -6.25000000e-04 -1.50000000e-03  5.93750000e-04 -1.19791667e-03
 -3.95833333e-04 -8.64583333e-04 -2.08333333e-05  7.87916667e-02
  7.50625000e-02  3.76354167e-02 -2.77291667e-02 -1.11250000e-02
  1.89687500e-02 -6.97916667e-04 -1.86458333e-03  5.83333333e-04
 -4.37500000e-04 -1.56250000e-04 -9.16666667e-04  1.86458333e-03
 -3.57291667e-03  1.16666667e-03  1.21875000e-03 -1.85416667e-03
  1.57291667e-03 -7.39583333e-04 -2.29166667e-03  4.37500000e-04


Overlap [0.70391667 0.00142009]
Gradient [-2.79166667e-03  8.43750000e-04 -2.08333333e-04 -1.12500000e-03
  1.29166667e-03 -1.41666667e-03 -4.27083333e-04  1.53125000e-03
  2.39583333e-04 -1.40625000e-03 -5.00000000e-04  1.70625000e-02
  1.83645833e-02  2.34062500e-02 -2.15729167e-02 -1.13020833e-02
  1.80729167e-02 -5.10416667e-04 -3.12500000e-05 -3.85416667e-04
 -1.36458333e-03  5.31250000e-04 -2.85416667e-03 -2.08333333e-05
 -1.20833333e-03 -1.22916667e-03 -1.35416667e-04 -5.41666667e-04
 -2.08333333e-04  1.45833333e-04 -6.25000000e-04 -2.01041667e-03
  4.16666667e-05  1.63750000e-02  1.88437500e-02  2.31562500e-02
 -2.39270833e-02 -1.13229167e-02  1.66979167e-02]
Shift optimizing step: 15
Overlap [0.71029167 0.00142063]
Gradient [ 1.46875000e-03 -1.77083333e-04 -1.44791667e-03  6.04166667e-04
 -5.83333333e-04  2.55208333e-03  9.68750000e-04 -1.04166667e-05
  7.60416667e-04  1.87500000e-04 -6.56250000e-04  1.39166667e-02
  1.61041667e-02  2.20833333e-02 -2.14375000e-02 -1.02291667e-

Overlap [0.7314375  0.00155774]
Gradient [ 1.35416667e-03  3.75000000e-04 -1.43750000e-03 -1.87500000e-04
  1.22916667e-03 -1.08333333e-03  6.25000000e-05  2.08333333e-05
 -6.87500000e-04  1.56250000e-04  2.77083333e-03  2.41666667e-03
  3.50000000e-03  8.56250000e-03 -1.41875000e-02 -1.04687500e-02
  1.35729167e-02  8.75000000e-04 -3.64583333e-04  3.54166667e-04
 -2.39583333e-03  2.39583333e-03 -4.68750000e-04 -5.83333333e-04
  6.25000000e-05 -1.77083333e-03  1.04166667e-04 -4.06250000e-04
 -3.17708333e-03 -1.15625000e-03 -3.06250000e-03 -6.45833333e-04
  8.33333333e-04  3.05208333e-03  1.09375000e-03  1.21875000e-02
 -1.22395833e-02 -1.03750000e-02  1.40312500e-02]
Shift optimizing step: 27
Overlap [0.734      0.00157402]
Gradient [-0.00229167  0.00036458  0.00230208 -0.00094792 -0.00170833  0.00226042
 -0.00014583 -0.00032292  0.00070833  0.00136458  0.00189583  0.00253125
  0.00386458  0.011375   -0.01214583 -0.00883333  0.01632292 -0.00135417
 -0.00177083 -0.00142708  0.00070833 -

Overlap [0.73922917 0.0017051 ]
Gradient [-2.32291667e-03 -2.29166667e-04  6.25000000e-04 -1.46875000e-03
 -1.05208333e-03  3.33333333e-03  6.25000000e-05  1.32291667e-03
  1.45833333e-03  1.52083333e-03  4.59375000e-03  2.50000000e-04
 -7.29166667e-04  3.25000000e-03 -5.05208333e-03 -6.94791667e-03
  1.09687500e-02 -1.82291667e-03 -1.67708333e-03  1.13541667e-03
 -1.32291667e-03  1.72916667e-03  1.77083333e-04 -2.18750000e-04
 -1.36458333e-03  1.72916667e-03  9.79166667e-04  1.89583333e-03
  9.79166667e-04  9.68750000e-04  2.08333333e-04  8.85416667e-04
  1.48958333e-03  3.95833333e-04  2.14583333e-03  1.85416667e-03
 -6.68750000e-03 -6.42708333e-03  1.10937500e-02]
Shift optimizing step: 40
Overlap [0.74491667 0.00169898]
Gradient [-1.54166667e-03  1.00000000e-03  1.14583333e-04 -1.97916667e-04
 -1.84375000e-03  1.97916667e-03 -3.54166667e-04 -1.71875000e-03
 -3.43750000e-04  2.60416667e-04  4.10416667e-03 -8.54166667e-04
  2.29166667e-04  7.29166667e-04 -6.22916667e-03 -7.63541667e-

Overlap [0.74733333 0.00181118]
Gradient [-8.95833333e-04 -2.81250000e-04 -1.76041667e-03 -1.13541667e-03
 -2.09375000e-03  6.03125000e-03  8.33333333e-05 -7.29166667e-04
 -1.81250000e-03  8.33333333e-04  4.11458333e-03 -8.33333333e-04
  3.85416667e-04  7.91666667e-04 -2.34375000e-03 -6.13541667e-03
  7.11458333e-03 -2.81250000e-04 -2.88541667e-03  2.63541667e-03
 -3.54166667e-04  9.89583333e-04  1.96875000e-03 -8.22916667e-04
 -1.09375000e-03  9.37500000e-05  3.95833333e-04 -1.66666667e-04
 -1.98958333e-03  4.79166667e-04 -3.64583333e-04  2.81250000e-04
  5.79166667e-03  1.67708333e-03  1.78125000e-03 -9.89583333e-04
 -2.32291667e-03 -6.80208333e-03  6.56250000e-03]
Shift optimizing step: 53
Overlap [0.7470625  0.00181352]
Gradient [-0.0015625   0.00101042  0.00209375  0.00038542 -0.00144792  0.00848958
 -0.00027083  0.00019792 -0.00092708  0.00154167  0.00398958 -0.00077083
 -0.00203125  0.00261458 -0.00345833 -0.00755208  0.00563542  0.00132292
 -0.00158333  0.00094792  0.001       

...

Overlap [0.97722917 0.00110979]
Gradient [-7.70833333e-04 -1.08333333e-03  5.20833333e-04  5.52083333e-04
  3.75000000e-04  5.62500000e-04 -2.60416667e-04  3.43750000e-04
  9.27083333e-04  1.88541667e-03  1.38541667e-03 -3.02083333e-04
  9.47916667e-04 -9.89583333e-04 -1.21875000e-03 -7.08333333e-04
  1.25000000e-04  2.73958333e-03 -1.85416667e-03 -1.29166667e-03
  5.10416667e-04  4.68750000e-04  1.68750000e-03  5.20833333e-05
  1.36458333e-03  9.06250000e-04  3.12500000e-05 -2.81250000e-04
  1.87500000e-04  2.09375000e-03  6.25000000e-04 -1.11458333e-03
 -3.26041667e-03  4.58333333e-04 -5.93750000e-04  2.89583333e-03
 -8.85416667e-04 -5.20833333e-05 -2.85416667e-03]
Shift optimizing step: 952
Overlap [0.98029167 0.00102022]
Gradient [-7.29166667e-05  9.37500000e-05  1.25000000e-04 -1.27083333e-03
 -1.97916667e-04  4.37500000e-04  2.50000000e-04  6.25000000e-05
  2.08333333e-04 -1.30208333e-03  1.69791667e-03  8.33333333e-04
  2.39583333e-03 -6.56250000e-04  8.85416667e-04  8.12500000e

Overlap [0.97989583 0.00104553]
Gradient [-1.46875000e-03  1.04166667e-04  3.43750000e-04 -1.22916667e-03
 -1.25000000e-04 -1.12500000e-03  9.06250000e-04 -4.89583333e-04
  1.82291667e-03  1.61458333e-03 -6.66666667e-04 -9.37500000e-05
  1.50000000e-03 -6.25000000e-05 -6.77083333e-04  5.31250000e-04
  2.10416667e-03  1.40625000e-03  5.93750000e-04  1.95833333e-03
  9.37500000e-05 -1.23958333e-03  2.29166667e-04 -1.11458333e-03
  7.81250000e-04  2.38541667e-03  8.95833333e-04 -2.08333333e-05
 -1.09375000e-03 -1.73958333e-03 -6.97916667e-04  1.43750000e-03
 -3.95833333e-04  8.12500000e-04  1.11022302e-16 -3.85416667e-04
  4.16666667e-05 -1.09375000e-03  3.15625000e-03]
Shift optimizing step: 964
Overlap [0.97920833 0.00107196]
Gradient [-7.70833333e-04  3.43750000e-04 -6.77083333e-04  1.32291667e-03
 -1.63541667e-03 -1.00000000e-03 -1.02083333e-03  3.12500000e-05
  1.54166667e-03 -1.69791667e-03  2.59375000e-03  3.43750000e-04
  1.04166667e-03 -3.12500000e-05  3.54166667e-04  5.00000000e

Overlap [0.97985417 0.00104916]
Gradient [-2.50000000e-04  3.22916667e-04  8.33333333e-04  1.35416667e-04
  3.22916667e-04 -2.70833333e-04 -5.72916667e-04 -1.47916667e-03
 -1.17708333e-03 -5.72916667e-04  2.11458333e-03 -6.66666667e-04
  1.07291667e-03  5.52083333e-04  1.38541667e-03  1.33333333e-03
 -1.92708333e-03  5.41666667e-04 -6.66666667e-04 -1.65625000e-03
  1.69791667e-03 -3.02083333e-04 -2.04166667e-03 -1.08333333e-03
 -1.44791667e-03 -1.50000000e-03  4.68750000e-04 -1.14583333e-04
 -2.26041667e-03 -2.66666667e-03 -5.93750000e-04 -3.08333333e-03
  8.33333333e-05 -6.97916667e-04 -2.70833333e-04 -6.45833333e-04
 -6.66666667e-04 -2.07291667e-03 -7.29166667e-05]
Shift optimizing step: 977
Overlap [0.97952083 0.0010529 ]
Gradient [-7.91666667e-04  7.70833333e-04  1.44791667e-03 -3.02083333e-04
 -1.22916667e-03  3.02083333e-04  2.08333333e-05 -2.08333333e-05
 -8.95833333e-04  3.85416667e-04 -9.68750000e-04  6.56250000e-04
  4.37500000e-04 -8.12500000e-04  6.56250000e-04  4.27083333e

Overlap [0.97958333 0.0010631 ]
Gradient [ 2.29166667e-04 -7.29166667e-04  1.50000000e-03 -1.13541667e-03
  4.68750000e-04  4.47916667e-04 -1.67708333e-03 -1.98958333e-03
  7.60416667e-04 -1.87500000e-04  1.10416667e-03  2.60416667e-04
  5.41666667e-04 -5.41666667e-04  2.79166667e-03 -4.89583333e-04
  4.58333333e-04 -1.17708333e-03  3.30208333e-03 -1.02083333e-03
  2.20833333e-03  3.18750000e-03  1.87500000e-04  2.32291667e-03
 -2.29166667e-04  2.04166667e-03  2.69791667e-03 -9.37500000e-05
 -4.27083333e-04  7.81250000e-04  2.83333333e-03 -6.97916667e-04
 -2.18750000e-04 -3.12500000e-05 -1.47916667e-03  1.16666667e-03
 -2.58333333e-03  2.95833333e-03  1.55208333e-03]
Shift optimizing step: 989
Overlap [0.9789375  0.00107069]
Gradient [-4.06250000e-04 -2.81250000e-04  4.16666667e-04 -1.06250000e-03
  1.60416667e-03  9.37500000e-05  1.30208333e-03  2.33333333e-03
 -5.62500000e-04 -2.60416667e-03  1.04166667e-03  2.70833333e-04
 -6.25000000e-05 -2.91666667e-04 -7.29166667e-05  5.72916667e

Overlap [0.97816667 0.00108827]
Gradient [-1.58333333e-03  2.08333333e-05  1.11458333e-03 -1.95833333e-03
 -7.39583333e-04  3.95833333e-04  3.85416667e-04 -8.33333333e-05
 -3.12500000e-04 -9.68750000e-04 -1.04166667e-04 -9.68750000e-04
  3.02083333e-04 -9.16666667e-04 -2.45833333e-03 -2.51041667e-03
 -5.93750000e-04 -4.89583333e-04 -1.97916667e-04 -5.41666667e-04
 -1.58333333e-03 -7.08333333e-04 -1.85416667e-03 -1.27083333e-03
  2.61458333e-03  3.85416667e-04  1.22916667e-03 -8.33333333e-05
 -2.08333333e-05 -1.14583333e-03  2.58333333e-03  1.14583333e-03
  9.06250000e-04 -3.12500000e-05 -2.14583333e-03 -1.35416667e-03
 -3.10416667e-03 -2.45833333e-03 -2.70833333e-04]

---------------------------------- 

Shift after optimizing: [-1.44229167e-01  7.08333333e-04  3.41229167e-01  6.29937500e-01
 -1.45268750e+00  1.58956250e+00  4.55729167e-02  8.12083333e-02
  1.36229167e-01  6.94843750e-01  9.17083333e-01  8.44843750e-01
  1.50292708e+00  1.48904167e+00 -1.48572917e+00 -2.24354167e-01
 -